# Text Analysis and Preprocessing Pipeline

## Notebook Overview

This notebook provides a comprehensive workflow for text data analysis and preprocessing.

- **Data Loading**: Set up the environment, load and explore text datasets, perform statistical analysis, and create reusable data loading functions
- **Preprocessing**: Apply various NLP preprocessing techniques using NLTK and SpaCy, analyze the impact of different methods, and develop a reusable preprocessing pipeline
- **Dataset Creation**: Create Multi-Label and Single-Label datasets for use in subsequent labs (Lab 4, Lab 5)

## Output Files

This notebook creates the following datasets for use in Lab 4 and Lab 5:

**Multi-Label Datasets:**
- `../Data/multi_label/tweets_preprocessed_train.parquet`
- `../Data/multi_label/tweets_preprocessed_test.parquet`
- `../Data/multi_label/tweets_preprocessed_validation.parquet`

**Single-Label Datasets (using Claude Haiku for intelligent label assignment):**
- `../Data/single_label/tweets_single_label_train.parquet`
- `../Data/single_label/tweets_single_label_test.parquet`
- `../Data/single_label/tweets_single_label_validation.parquet`

---

## 1. Data Loading

This section covers the initial setup and data loading phase. We will install necessary libraries, load the dataset, explore its structure and characteristics, perform statistical analysis, filter the data to text-only content, and create reusable functions for future use.

### 1.1 Load and Explore Dataset

Loading the dataset into a pandas DataFrame and performing initial exploration. This includes viewing the first few rows, checking data types, examining the shape of the dataset, and identifying any missing values.

#### 1.1.1 Basic Dataset Overview

In [ ]:
import pandas as pd
from datasets import load_dataset

# Load the dataset directly from Hugging Face using the datasets library
dataset = load_dataset("cardiffnlp/tweet_topic_multi", split="train_all")

# Convert to pandas DataFrame
df = dataset.to_pandas()

# Display basic information
print("Row count, Column count:", df.shape)
print("\nColumn Names:", df.columns.tolist())
print("\nData Types:")
print(df.dtypes)
print("\nMissing Values:")
print(df.isnull().sum())
print("\nFirst 10 Rows:")
display(df.head(10))

#### 1.1.2 Initial Data Exploration with NLTK

Using NLTK to perform basic text exploration and tokenization on a sample of tweets.

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
import string

# Download required NLTK data
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('punkt_tab', quiet=True)

print("\n" + "="*50)
print("INITIAL TEXT EXPLORATION WITH NLTK")
print("="*50)

# Show sample tokenization on a few tweets
print("\nSample Tweet Tokenization:")
sample_text = df['text'].iloc[0]
print(f"Original: {sample_text}")
tokens = word_tokenize(sample_text)
print(f"Tokens: {tokens}")

# Get English stopwords
stop_words = set(stopwords.words('english'))
print(f"\nNumber of English stopwords: {len(stop_words)}")
print(f"Sample stopwords: {list(stop_words)[:10]}")

#### 1.1.3 Initial Data Exploration with spaCy

Using spaCy to perform basic linguistic analysis on sample tweets.

In [ ]:
import spacy

# Load spaCy model
try:
    nlp = spacy.load('en_core_web_sm')
except:
    print("Downloading spaCy model...")
    import subprocess
    subprocess.run(['python', '-m', 'spacy', 'download', 'en_core_web_sm'])
    nlp = spacy.load('en_core_web_sm')

print("\n" + "="*50)
print("INITIAL TEXT EXPLORATION WITH spaCy")
print("="*50)

# Analyze a sample tweet
sample_text = df['text'].iloc[5]
doc = nlp(sample_text)

print(f"\nSample Tweet: {sample_text}")
print(f"\nTokens: {[token.text for token in doc]}")
print(f"\nPOS Tags: {[(token.text, token.pos_) for token in doc][:10]}")
print(f"\nEntities: {[(ent.text, ent.label_) for ent in doc.ents]}")

### 1.2 Calculate Statistics

Computing descriptive statistics on the dataset to understand the data distribution. This includes category distributions, temporal patterns, duplicate analysis, text length statistics, word frequency analysis, and named entity statistics using pandas and NLP libraries.

#### 1.2.1 Category/Topic Distribution Analysis

Analyzing how many tweets are available for each topic category using pandas operations.

In [ ]:
from collections import Counter

# Convert label_name to lists (they're already arrays, just ensure they're lists)
df['topics_list'] = df['label_name'].apply(lambda x: list(x) if not isinstance(x, list) else x)

# Count tweets per category
all_topics = []
for topics in df['topics_list']:
    all_topics.extend(topics)

topic_counts = Counter(all_topics)

# Create a DataFrame for better visualization
topic_df = pd.DataFrame(
    topic_counts.items(), 
    columns=['Topic', 'Count']
).sort_values('Count', ascending=False)

print("\n" + "="*50)
print("TWEETS PER CATEGORY")
print("="*50)
print(topic_df.to_string(index=False))
print(f"\nTotal unique topics: {len(topic_counts)}")

# Multi-label statistics
df['num_topics'] = df['topics_list'].apply(len)
print(f"\nTweets with 1 topic: {(df['num_topics'] == 1).sum()} ({(df['num_topics'] == 1).sum() / len(df) * 100:.1f}%)")
print(f"Tweets with 2+ topics: {(df['num_topics'] > 1).sum()} ({(df['num_topics'] > 1).sum() / len(df) * 100:.1f}%)")
print(f"\nMax topics per tweet: {df['num_topics'].max()}")
print(f"Average topics per tweet: {df['num_topics'].mean():.2f}")

#### 1.2.1.1 Single-Label vs Multi-Label Analysis per Topic

Analyzing which labels appear as single labels (sole topic of a tweet) versus only appearing in multi-label tweets (always combined with other topics). This is important for understanding class imbalance in single-label classification.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

print("\n" + "="*60)
print("SINGLE-LABEL vs MULTI-LABEL ANALYSIS PER TOPIC")
print("="*60)

# Separate single-label and multi-label tweets
single_label_tweets = df[df['num_topics'] == 1]
multi_label_tweets = df[df['num_topics'] > 1]

print(f"\nTotal tweets: {len(df)}")
print(f"Single-label tweets: {len(single_label_tweets)} ({len(single_label_tweets)/len(df)*100:.1f}%)")
print(f"Multi-label tweets: {len(multi_label_tweets)} ({len(multi_label_tweets)/len(df)*100:.1f}%)")

# Count occurrences in single-label vs multi-label tweets for each topic
single_label_counts = Counter()
multi_label_counts = Counter()

for topics in single_label_tweets['topics_list']:
    for topic in topics:
        single_label_counts[topic] += 1

for topics in multi_label_tweets['topics_list']:
    for topic in topics:
        multi_label_counts[topic] += 1

# Create analysis DataFrame
analysis_data = []
all_topics_set = set(single_label_counts.keys()) | set(multi_label_counts.keys())

for topic in all_topics_set:
    single_count = single_label_counts.get(topic, 0)
    multi_count = multi_label_counts.get(topic, 0)
    total = single_count + multi_count
    single_pct = (single_count / total * 100) if total > 0 else 0
    
    analysis_data.append({
        'Topic': topic,
        'Single-Label': single_count,
        'Multi-Label': multi_count,
        'Total': total,
        'Single-Label %': single_pct,
        'Status': '⚠️ ONLY MULTI-LABEL' if single_count == 0 else ('✓ Has Single-Label' if single_count > 50 else '⚠️ Few Single-Labels')
    })

analysis_df = pd.DataFrame(analysis_data).sort_values('Total', ascending=True)

# Create grouped bar chart
fig, ax = plt.subplots(figsize=(14, 10))

# Data for plotting
topics = analysis_df['Topic'].tolist()
single_counts = analysis_df['Single-Label'].tolist()
multi_counts = analysis_df['Multi-Label'].tolist()

y_pos = np.arange(len(topics))
bar_height = 0.35

# Create bars
bars_single = ax.barh(y_pos - bar_height/2, single_counts, bar_height, 
                       label='Single-Label', color='#3498db', edgecolor='black', linewidth=0.5)
bars_multi = ax.barh(y_pos + bar_height/2, multi_counts, bar_height, 
                      label='Multi-Label', color='#e74c3c', edgecolor='black', linewidth=0.5)

# Add count annotations
for i, (single, multi) in enumerate(zip(single_counts, multi_counts)):
    ax.annotate(f'{single}', xy=(single + 10, i - bar_height/2), va='center', ha='left', fontsize=9, color='#2980b9')
    ax.annotate(f'{multi}', xy=(multi + 10, i + bar_height/2), va='center', ha='left', fontsize=9, color='#c0392b')

# Customize chart
ax.set_yticks(y_pos)
ax.set_yticklabels(topics)
ax.set_xlabel('Number of Tweets', fontsize=12)
ax.set_ylabel('Topic', fontsize=12)
ax.set_title('Single-Label vs Multi-Label Tweet Distribution per Topic\n(Blue: single label | Red: multi label)', fontsize=14)
ax.legend(loc='lower right', fontsize=11)
ax.set_xlim(0, max(max(single_counts), max(multi_counts)) * 1.15)

# Add gridlines for better readability
ax.xaxis.grid(True, linestyle='--', alpha=0.7)
ax.set_axisbelow(True)

plt.tight_layout()
plt.show()

# Print table
print("\n" + "-"*60)
print("Label Distribution: Single-Label vs Multi-Label Tweets")
print("-"*60)
print(analysis_df.sort_values('Single-Label', ascending=True).to_string(index=False))

# Highlight problematic labels
only_multi_labels = analysis_df[analysis_df['Single-Label'] == 0]['Topic'].tolist()
few_single_labels = analysis_df[(analysis_df['Single-Label'] > 0) & (analysis_df['Single-Label'] < 50)]['Topic'].tolist()

print("\n" + "="*60)
print("⚠️ POTENTIAL ISSUES FOR SINGLE-LABEL CLASSIFICATION")
print("="*60)

if only_multi_labels:
    print(f"\n🚨 Labels that NEVER appear as single-label ({len(only_multi_labels)}):")
    print("   These labels will have 0 training samples in single-label mode!")
    for label in only_multi_labels:
        total = analysis_df[analysis_df['Topic'] == label]['Total'].values[0]
        print(f"   - {label} (total: {total}, always combined with other labels)")
else:
    print("\n✓ All labels have at least some single-label tweets")

if few_single_labels:
    print(f"\n⚠️ Labels with very few single-label tweets ({len(few_single_labels)}):")
    print("   These labels may be underrepresented in single-label classification!")
    for label in few_single_labels:
        single = analysis_df[analysis_df['Topic'] == label]['Single-Label'].values[0]
        total = analysis_df[analysis_df['Topic'] == label]['Total'].values[0]
        print(f"   - {label} (single: {single}, total: {total}, {single/total*100:.1f}% as single-label)")

print("\n" + "="*60)

#### 1.2.2 Temporal Distribution Analysis

Analyzing the distribution of tweets across different time periods using pandas datetime operations.

In [ ]:
# Convert date column to datetime
df['date_parsed'] = pd.to_datetime(df['date'])
df['year'] = df['date_parsed'].dt.year
df['month'] = df['date_parsed'].dt.month
df['day_of_week'] = df['date_parsed'].dt.day_name()

# Count tweets per year
year_counts = df['year'].value_counts().sort_index()

print("\n" + "="*50)
print("TEMPORAL DISTRIBUTION STATISTICS")
print("="*50)

print("\nTweets per Year:")
for year, count in year_counts.items():
    print(f"  {year}: {count:,} tweets ({count/len(df)*100:.1f}%)")

print(f"\nDate range: {df['date_parsed'].min().date()} to {df['date_parsed'].max().date()}")
print(f"Total days covered: {(df['date_parsed'].max() - df['date_parsed'].min()).days} days")

# Month distribution
print("\nTweets per Month (across all years):")
month_counts = df['month'].value_counts().sort_index()
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for month, count in month_counts.items():
    print(f"  {month_names[month-1]}: {count:,} tweets")

# Day of week distribution
print("\nTweets per Day of Week:")
dow_counts = df['day_of_week'].value_counts()
for day, count in dow_counts.items():
    print(f"  {day}: {count:,} tweets ({count/len(df)*100:.1f}%)")

#### 1.2.3 Duplicate Detection Statistics

Identifying and quantifying exact and potential near-duplicate tweets using pandas string operations.

In [ ]:
print("\n" + "="*50)
print("DUPLICATE ANALYSIS")
print("="*50)

# Check for exact text duplicates
duplicate_texts = df['text'].duplicated().sum()
print(f"Exact duplicate tweets: {duplicate_texts} ({duplicate_texts / len(df) * 100:.2f}%)")

# Check for duplicate IDs
duplicate_ids = df['id'].duplicated().sum()
print(f"Duplicate IDs: {duplicate_ids}")

# Show sample duplicates if any exist
if duplicate_texts > 0:
    print("\nSample Duplicate Tweets:")
    duplicated_mask = df['text'].duplicated(keep=False)
    sample_duplicates = df[duplicated_mask].groupby('text').head(2)
    print(sample_duplicates[['text', 'date', 'label_name']].head(6).to_string())
else:
    print("\n✓ No exact duplicate tweets found!")

# Near-duplicate detection (same first 50 characters)
df['text_start'] = df['text'].str.lower().str.strip().str[:50]
potential_near_dupes = df['text_start'].duplicated().sum()
print(f"\nPotential near-duplicates (same first 50 chars): {potential_near_dupes} ({potential_near_dupes / len(df) * 100:.2f}%)")

# List all near-duplicate IDs grouped
if potential_near_dupes > 0:
    print("\nAll Near-Duplicate Groups (sorted by group size):")
    near_dupe_mask = df['text_start'].duplicated(keep=False)
    near_dupe_df = df[near_dupe_mask][['text_start', 'id', 'text']]
    
    # Group by text_start and collect IDs
    grouped = near_dupe_df.groupby('text_start')
    
    # Sort groups by size (largest first)
    group_sizes = grouped.size().sort_values(ascending=False)
    
    print(f"\nTotal near-duplicate groups: {len(group_sizes)}")
    print(f"Largest group size: {group_sizes.max()}\n")
    
    for i, (text_start, size) in enumerate(group_sizes.items(), 1):
        group_ids = grouped.get_group(text_start)['id'].tolist()
        print(f"Group {i} (Size: {size}):")
        print(f"  Text preview: {text_start}...")
        print(f"  IDs: {group_ids}")
        print()
else:
    print("\n✓ No potential near-duplicates found!")

#### 1.2.4 Text Length and Composition Statistics

Analyzing text characteristics using pandas string methods and aggregations.

In [ ]:
# Calculate text statistics
df['text_length'] = df['text'].str.len()
df['word_count'] = df['text'].str.split().str.len()
df['avg_word_length'] = df['text'].str.replace(' ', '').str.len() / df['word_count']

# Count special characters
df['hashtag_count'] = df['text'].str.count('#')
df['mention_count'] = df['text'].str.count('@')
df['url_count'] = df['text'].str.count('http')

print("\n" + "="*50)
print("TEXT LENGTH AND COMPOSITION STATISTICS")
print("="*50)

print("\nCharacter Length Statistics:")
print(f"  Mean: {df['text_length'].mean():.1f} characters")
print(f"  Median: {df['text_length'].median():.1f} characters")
print(f"  Std Dev: {df['text_length'].std():.1f} characters")
print(f"  Min: {df['text_length'].min()} characters")
print(f"  Max: {df['text_length'].max()} characters")

print("\nWord Count Statistics:")
print(f"  Mean: {df['word_count'].mean():.1f} words")
print(f"  Median: {df['word_count'].median():.1f} words")
print(f"  Std Dev: {df['word_count'].std():.1f} words")
print(f"  Min: {df['word_count'].min()} words")
print(f"  Max: {df['word_count'].max()} words")

print("\nAverage Word Length:")
print(f"  Mean: {df['avg_word_length'].mean():.2f} characters per word")
print(f"  Median: {df['avg_word_length'].median():.2f} characters per word")

print("\nSpecial Character Statistics:")
print(f"  Tweets with hashtags: {(df['hashtag_count'] > 0).sum()} ({(df['hashtag_count'] > 0).sum()/len(df)*100:.1f}%)")
print(f"  Average hashtags per tweet: {df['hashtag_count'].mean():.2f}")
print(f"  Tweets with mentions: {(df['mention_count'] > 0).sum()} ({(df['mention_count'] > 0).sum()/len(df)*100:.1f}%)")
print(f"  Average mentions per tweet: {df['mention_count'].mean():.2f}")
print(f"  Tweets with URLs: {(df['url_count'] > 0).sum()} ({(df['url_count'] > 0).sum()/len(df)*100:.1f}%)")

# Show extreme examples
print("\nExtreme Examples:")
print(f"\nShortest tweet ({df['text_length'].min()} chars):")
print(f"  {df.loc[df['text_length'].idxmin(), 'text']}")
print(f"\nLongest tweet ({df['text_length'].max()} chars, first 150):")
print(f"  {df.loc[df['text_length'].idxmax(), 'text'][:150]}...")

#### 1.2.5 Word Frequency Statistics (NLTK)

Analyzing word frequency patterns using NLTK tokenization and pandas aggregations.

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

# Get English stopwords
stop_words = set(stopwords.words('english'))

print("\n" + "="*50)
print("WORD FREQUENCY STATISTICS (NLTK)")
print("="*50)

# Tokenize all tweets
all_tokens = []
for text in df['text']:
    tokens = word_tokenize(text.lower())
    # Remove punctuation and stopwords, keep only meaningful words
    tokens = [
        t for t in tokens 
        if t not in string.punctuation 
        and t not in stop_words 
        and len(t) > 2
    ]
    all_tokens.extend(tokens)

token_freq = Counter(all_tokens)
top_words = pd.DataFrame(
    token_freq.most_common(30), 
    columns=['Word', 'Frequency']
)

print(f"\nVocabulary Statistics:")
print(f"  Total tokens (after filtering): {len(all_tokens):,}")
print(f"  Unique words (vocabulary size): {len(token_freq):,}")
print(f"  Average token frequency: {len(all_tokens) / len(token_freq):.2f}")
print(f"  Words appearing only once: {sum(1 for count in token_freq.values() if count == 1):,}")
print(f"  Words appearing 10+ times: {sum(1 for count in token_freq.values() if count >= 10):,}")

print(f"\nTop 30 Most Common Words:")
print(top_words.to_string(index=False))

#### 1.2.6 Named Entity Statistics (spaCy)

Analyzing named entity distributions using spaCy NER and pandas aggregations.

In [ ]:
from tqdm import tqdm

print("\n" + "="*50)
print("NAMED ENTITY STATISTICS (spaCy)")
print("="*50)
print(f"\nAnalyzing {len(df)} tweets... This may take a few minutes...")

# Extract entities from all tweets
all_entities = []
tweets_with_entities = 0
entity_counts_per_tweet = []

# Use nlp.pipe for better performance
texts = df['text'].apply(lambda x: x[:500]).tolist()

for doc in tqdm(nlp.pipe(texts, batch_size=50), total=len(df), desc="Processing"):
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    entity_counts_per_tweet.append(len(entities))
    if entities:
        tweets_with_entities += 1
    all_entities.extend(entities)

# Add entity count to dataframe
df['entity_count'] = entity_counts_per_tweet

print(f"\nEntity Detection Statistics:")
print(f"  Tweets with entities: {tweets_with_entities:,} ({tweets_with_entities/len(df)*100:.1f}%)")
print(f"  Tweets without entities: {len(df) - tweets_with_entities:,} ({(len(df) - tweets_with_entities)/len(df)*100:.1f}%)")
print(f"  Total entities found: {len(all_entities):,}")
print(f"  Average entities per tweet: {df['entity_count'].mean():.2f}")
print(f"  Median entities per tweet: {df['entity_count'].median():.0f}")
print(f"  Max entities in a tweet: {df['entity_count'].max()}")

# Count entity types
entity_types = Counter([label for text, label in all_entities])
entity_type_df = pd.DataFrame(
    entity_types.most_common(), 
    columns=['Entity Type', 'Count']
)
entity_type_df['Percentage'] = (entity_type_df['Count'] / len(all_entities) * 100).round(1)

print("\nEntity Type Distribution:")
print(entity_type_df.to_string(index=False))

print("\nEntity Type Legend:")
print("\nEntity Type Legend (Complete):")
print("  PERSON      = People, including fictional characters")
print("  ORG         = Companies, agencies, institutions, organizations")
print("  GPE         = Countries, cities, states (Geo-Political Entities)")
print("  DATE        = Absolute or relative dates or periods")
print("  CARDINAL    = Numerals that do not fall under another type")
print("  MONEY       = Monetary values, including unit")
print("  TIME        = Times smaller than a day")
print("  NORP        = Nationalities, religious or political groups")
print("  ORDINAL     = First, second, third, etc.")
print("  WORK_OF_ART = Titles of books, songs, movies, etc.")
print("  EVENT       = Named hurricanes, battles, wars, sports events")
print("  PRODUCT     = Objects, vehicles, foods, etc. (not services)")
print("  LOC         = Non-GPE locations, mountain ranges, bodies of water")
print("  FAC         = Buildings, airports, highways, bridges, etc.")
print("  QUANTITY    = Measurements, as of weight or distance")
print("  LAW         = Named documents made into laws")
print("  PERCENT     = Percentage, including '%'")
print("  LANGUAGE    = Any named language")

### 1.3 Filter to Text Only

Filtering the dataset to extract only text columns and remove any non-textual data. This step ensures that subsequent processing focuses exclusively on textual content and handles any data type conversions if necessary.

In [ ]:
def filter_to_text_only(dataframe, text_col='text', label_col='label_name'):
    """
    Filter dataset to only text and label columns, removing all numbers.
    
    Parameters:
    -----------
    dataframe : pd.DataFrame
        Input dataframe to filter
    text_col : str
        Name of the text column (default: 'text')
    label_col : str
        Name of the label column (default: 'label_name')
    
    Returns:
    --------
    pd.DataFrame
        Filtered dataframe with only text columns and no numbers
    """
    
    # Step 1: Select only the text and label_name columns
    df_filtered = dataframe[[text_col, label_col]].copy()
    
    # Step 2: Remove all numbers from the text column using regex
    # This removes all digits (0-9) from the text
    df_filtered[text_col] = df_filtered[text_col].str.replace(r'\d+', '', regex=True)
    
    # Step 3: Handle label_name - convert to string if it's a list, then remove numbers
    # First check if it's already a list or needs conversion
    if isinstance(df_filtered[label_col].iloc[0], list):
        # Keep as list, no number removal needed (labels are text)
        pass
    else:
        # If it's a string representation, convert and clean
        df_filtered[label_col] = df_filtered[label_col].astype(str)
    
    # Step 4: Clean up any extra whitespace created by removing numbers
    df_filtered[text_col] = df_filtered[text_col].str.replace(r'\s+', ' ', regex=True).str.strip()
    
    return df_filtered


# Apply the filtering function
print("\n" + "="*50)
print("FILTERING TO TEXT ONLY")
print("="*50)

# Show original dataset info
print("\nOriginal Dataset:")
print(f"  Shape: {df.shape}")
print(f"  Columns: {df.columns.tolist()}")
print(f"  Sample text: {df['text'].iloc[0][:100]}...")

# Apply the filter
df_text_only = filter_to_text_only(df)

# Show filtered dataset info
print("\nFiltered Dataset (Text Only - No Numbers):")
print(f"  Shape: {df_text_only.shape}")
print(f"  Columns: {df_text_only.columns.tolist()}")
print(f"  Sample text: {df_text_only['text'].iloc[0][:100]}...")

# Show examples of number removal
print("\nExamples of Number Removal:")
for i in range(3):
    print(f"\nExample {i+1}:")
    print(f"  Original:  {df['text'].iloc[i][:80]}...")
    print(f"  Filtered:  {df_text_only['text'].iloc[i][:80]}...")

# Show label_name comparison
print("\nLabel Name Comparison:")
print(f"  Original label_name: {df['label_name'].iloc[0]}")
print(f"  Filtered label_name: {df_text_only['label_name'].iloc[0]}")

# Statistics on number removal
original_chars = df['text'].str.len().sum()
filtered_chars = df_text_only['text'].str.len().sum()
chars_removed = original_chars - filtered_chars

print(f"\nCharacter Statistics:")
print(f"  Original total characters: {original_chars:,}")
print(f"  Filtered total characters: {filtered_chars:,}")
print(f"  Characters removed (numbers): {chars_removed:,} ({chars_removed/original_chars*100:.2f}%)")

print("\n✓ Dataset successfully filtered to text only (numbers removed)")

### 1.4 Create Reusable Loading and Filtering Functions

Reusable functions that encapsulate the data loading and text-filtering logic.

In [ ]:
import pandas as pd
from datasets import load_dataset

# Load the dataset directly from Hugging Face using the datasets library
dataset = load_dataset("cardiffnlp/tweet_topic_multi", split="train_all")

# Convert to pandas DataFrame
tweets_raw = dataset.to_pandas()

# Filter the dataset to text only (no numbers)
def filter_to_text_only(dataframe, text_col='text', label_col='label_name', label_num_col='label'):
    df_filtered = dataframe[[text_col, label_col, label_num_col]].copy()
    df_filtered[text_col] = df_filtered[text_col].str.replace(r'\d+', '', regex=True)
    
    if isinstance(df_filtered[label_col].iloc[0], list):
        pass
    else:
        df_filtered[label_col] = df_filtered[label_col].astype(str)
    
    df_filtered[text_col] = df_filtered[text_col].str.replace(r'\s+', ' ', regex=True).str.strip()
    
    return df_filtered

tweets_text_only = filter_to_text_only(tweets_raw)

print("\n✓ Dataset successfully loaded and filtered to text only")

---

## 2. Preprocessing

This section focuses on text preprocessing techniques. We will review common preprocessing methods, apply them systematically using NLTK and SpaCy, analyze how the order of operations affects results, evaluate the usefulness of each method for different scenarios, and create a reusable preprocessing pipeline.


### Methods Used (in Order)

#### 1. **Remove RT indicator**
   - **Purpose:** Removes retweet markers that don't carry semantic meaning
   - **Order:** FIRST - removes noise before any text processing

#### 2. **Remove placeholders (USERNAME, URL, mentions)**
   - **Purpose:** Removes dataset-specific placeholders that don't contribute to topic classification
   - **Order:** EARLY - clean structural noise before text normalization

#### 3. **Convert emojis to text**
   - **Purpose:** Transforms emojis into descriptive words that preserve semantic meaning (🎮 → video game)
   - **Order:** AFTER placeholders, BEFORE hashtags - emojis may appear anywhere in text

#### 4. **Extract hashtag text**
   - **Purpose:** Preserves topic-relevant keywords from hashtags (#Gaming → Gaming)
   - **Order:** AFTER emoji conversion - hashtags rarely contain emojis but order matters for consistency

#### 5. **Segment CamelCase words**
   - **Purpose:** Splits compound words for better tokenization (GameOfThrones → Game Of Thrones)
   - **Order:** BEFORE lowercase - capitalization patterns guide segmentation

#### 6. **Normalize whitespace and lowercase**
   - **Purpose:** Standardizes text format for consistent processing
   - **Order:** AFTER special token handling - ensures uniform text before tokenization

#### 7. **Tokenize with SpaCy**
   - **Purpose:** Splits text into linguistic tokens with POS and lemma information
   - **Order:** AFTER normalization - requires clean, lowercase text

#### 8. **Filter and lemmatize tokens**
   - **Purpose:** Removes noise tokens and reduces words to base forms
   - **Order:** LAST - operates on clean, filtered tokens

### Why did we choose this order?

**Critical Dependencies:**
- Emoji conversion **before** lowercase → emoji descriptions use underscores that get normalized
- Special tokens **before** tokenization → removes as complete units
- Lowercase **before** tokenization → consistent stopword matching
- Tokenization **before** filtering → need tokens to filter
- Lemmatization **last** → final transformation on clean tokens

### 2.1 Reusable Preprocessing Pipeline

Development of a modular, configurable preprocessing function that can be easily reused in future labs. The pipeline allows for flexible selection of preprocessing steps and parameters, making it adaptable to different text analysis tasks and requirements.

In [ ]:
# Download required NLTK data
import nltk
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt_tab', quiet=True)

# Load SpaCy model
import spacy
try:
    nlp = spacy.load('en_core_web_sm')
    print("✓ SpaCy model loaded successfully")
except:
    print("Installing SpaCy model...")
    import os
    os.system('python -m spacy download en_core_web_sm')
    nlp = spacy.load('en_core_web_sm')
    print("✓ SpaCy model loaded successfully")

# Import emoji package for emoji handling
import emoji

def is_latin_alphabet(word):
    """
    Check if a word contains only Latin alphabet characters.
    Filters out words with Cyrillic, Arabic, Chinese, etc.
    """
    if not word:
        return False
    return all(ord('a') <= ord(c.lower()) <= ord('z') for c in word)

def segment_camelcase(text):
    """
    Segment CamelCase words into separate words without regex.
    Example: 'GameOfThrones' → 'Game Of Thrones'
    This is important for hashtags like #GameOfThrones after removing the #
    """
    if not text:
        return text
    
    result = []
    
    for i, char in enumerate(text):
        # Add current character
        result.append(char)
        
        # Check if we need to insert a space
        if i < len(text) - 1:
            current = char
            next_char = text[i + 1]
            
            # Case 1: lowercase → uppercase (e.g., 'e' → 'O' in 'GameOf')
            if current.islower() and next_char.isupper():
                result.append(' ')
            
            # Case 2: uppercase → uppercase → lowercase (e.g., 'HTML' → 'Parser')
            elif i < len(text) - 2:
                after_next = text[i + 2]
                if current.isupper() and next_char.isupper() and after_next.islower():
                    result.append(' ')
    
    return ''.join(result)

def preprocess_tweet(text):
    """
    Topic-optimized preprocessing for tweet classification.
    Preserves topic-relevant information while removing noise.
    Removes special characters, emojis, and non-Latin script words.
    """
    if not isinstance(text, str):
        return ""
        
    # Step 1: Remove RT (retweet indicator)
    text = text.replace('RT ', ' ').replace('rt ', ' ')
    
    # Step 2: Remove URLs and placeholders
    text = text.replace('{{URL}}', ' ')
    text = text.replace('{{USERNAME}}', ' ')
    for protocol in ['https://', 'http://', 'www.']:
        if protocol in text:
            parts = text.split(protocol)
            text = parts[0] + ' ' + ' '.join([' '.join(p.split()[1:]) if p.split() else '' for p in parts[1:]])
    
    # Step 3: Remove mentions
    words_list = text.split()
    words_list = [w for w in words_list if not (w.startswith('{@') or w.startswith('@'))]
    text = ' '.join(words_list)
    
    # Step 3.5: Convert emojis to text descriptions (🎮 → video game)
    text = emoji.demojize(text, delimiters=(" ", " "))
    text = text.replace('_', ' ')
    
    # Step 4: Extract hashtag text (#Gaming → Gaming, #GameOfThrones → GameOfThrones)
    words_list = text.split()
    words_list = [w[1:] if w.startswith('#') else w for w in words_list]
    text = ' '.join(words_list)
    
    # Step 4.5: Segment CamelCase words
    # GameOfThrones → Game Of Thrones
    text = segment_camelcase(text)
    
    # Step 5: Normalize whitespace and lowercase
    text = ' '.join(text.split())
    text = text.lower()
    
    # Step 6: Tokenize with SpaCy
    doc = nlp(text)
    
    # Step 7: Filter and lemmatize tokens
    processed_tokens = []
    for token in doc:
        # Skip punctuation
        if token.is_punct:
            continue
        
        # Skip if not alphabetic (removes special characters, emojis, numbers)
        if not token.is_alpha:
            continue
        
        # Skip tokens shorter than 2 characters
        if len(token.text) < 2:
            continue
        
        # Remove stopwords (using SpaCy's stopword detection)
        if token.is_stop:
            continue
        
        # Check if word uses Latin alphabet (filters out Cyrillic, Arabic, Chinese, etc.)
        if not is_latin_alphabet(token.text):
            continue
        
        # Use lemmatized form
        processed_tokens.append(token.lemma_)
    
    return ' '.join(processed_tokens)

# Create a copy of the original dataframe
tweets_preprocessed_train = tweets_text_only.copy()

# Apply preprocessing
tweets_preprocessed_train['text'] = tweets_preprocessed_train['text'].apply(preprocess_tweet)

print("\n✓ Preprocessing complete!")
print(f"✓ Processed {len(tweets_preprocessed_train)} tweets")
print(f"✓ Original 'tweets_text_only' unchanged | Processed data in 'tweets_preprocessed_train'")

# Save the DataFrame to the Data folder
import os

# Create Data folder if it does not exist
os.makedirs('../Data', exist_ok=True)

# Save the data as parquet
output_path = '../Data/multi_label/tweets_preprocessed_train.parquet'
tweets_preprocessed_train.to_parquet(output_path, index=False)

print(f"✓ Shape: {tweets_preprocessed_train.shape}")
print(f"\n✓ Training DataFrame saved to: {output_path}")
print(f"✓ Features: {list(tweets_preprocessed_train.columns)}")

### 2.2 Preprocessing for Test and Validation Splits

Apply the same preprocessing pipeline to the test and validation splits from HuggingFace to ensure consistency between training and evaluation data.

In [ ]:
# Load test and validation splits from HuggingFace
print("Loading test and validation splits from HuggingFace...")
test_dataset = load_dataset("cardiffnlp/tweet_topic_multi", split="test_2021")
val_dataset = load_dataset("cardiffnlp/tweet_topic_multi", split="validation_2021")

# Convert to pandas DataFrames
test_raw = test_dataset.to_pandas()
val_raw = val_dataset.to_pandas()

print(f"Test samples: {len(test_raw):,}")
print(f"Validation samples: {len(val_raw):,}")

# Apply the same text-only filter (remove numbers)
test_text_only = filter_to_text_only(test_raw)
val_text_only = filter_to_text_only(val_raw)

# Apply the same preprocessing pipeline
tweets_preprocessed_test = test_text_only.copy()
tweets_preprocessed_test['text'] = tweets_preprocessed_test['text'].apply(preprocess_tweet)

tweets_preprocessed_validation = val_text_only.copy()
tweets_preprocessed_validation['text'] = tweets_preprocessed_validation['text'].apply(preprocess_tweet)

# Save test data as parquet
test_output_path = '../Data/multi_label/tweets_preprocessed_test.parquet'
tweets_preprocessed_test.to_parquet(test_output_path, index=False)

# Save validation data as parquet
val_output_path = '../Data/multi_label/tweets_preprocessed_validation.parquet'
tweets_preprocessed_validation.to_parquet(val_output_path, index=False)

print(f"\n✓ Test data preprocessing complete!")
print(f"✓ Saved to: {test_output_path}")
print(f"✓ Shape: {tweets_preprocessed_test.shape}")
print(f"✓ Features: {list(tweets_preprocessed_test.columns)}")

print(f"\n✓ Validation data preprocessing complete!")
print(f"✓ Saved to: {val_output_path}")
print(f"✓ Shape: {tweets_preprocessed_validation.shape}")
print(f"✓ Features: {list(tweets_preprocessed_validation.columns)}")

### 2.3 Prepare Data for Training

This section addresses a common issue in multi-label classification: **class imbalance**. Some labels in our dataset have very few training samples, which can negatively impact model performance during training.

**Why this matters:**
- Labels with too few samples cannot be learned effectively by the model
- Class imbalance can lead to biased predictions toward majority classes
- Having consistent labels across train/validation/test splits is crucial for proper evaluation

**What this function does:**
1. **Visualizes** the distribution of labels using matplotlib to identify potential imbalances
2. **Removes** labels with fewer than a specified threshold (default: 180 tweets) from all data splits
3. **Updates** the binary label vectors to reflect the removed labels
4. **Ensures consistency** by applying the same filtering to train, validation, and test sets

### 2.2.1 Label Co-occurrence Analysis

This section analyzes which labels frequently appear together in multi-label tweets. Understanding label co-occurrence patterns can provide insights into:
- **Topic relationships**: Which topics are semantically related or commonly discussed together
- **Dataset characteristics**: Natural clustering of content themes
- **Model design considerations**: Whether certain label combinations should be treated specially

For tweets with 3+ labels, we count all possible label pairs (e.g., for labels A, B, C, we count pairs: A-B, A-C, B-C).

In [ ]:
# ============================================================
# LABEL CO-OCCURRENCE ANALYSIS
# ============================================================
# Analysiert welche Label-Paare am häufigsten gemeinsam auftreten
# Bei Tweets mit 3+ Labels werden alle möglichen Paare gezählt

from itertools import combinations
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import re

def parse_label_names_cooc(label_str):
    """Parse the label_name string into a list of labels."""
    if isinstance(label_str, str):
        label_str = label_str.strip()
        if label_str.startswith('[') and label_str.endswith(']'):
            content = label_str[1:-1]
            items = re.findall(r"'([^']*)'", content)
            return items
    return []

# Verwende die gespeicherten preprocessed Daten
df_cooc = tweets_preprocessed_train.copy()
df_cooc['labels'] = df_cooc['label_name'].apply(parse_label_names_cooc)
df_cooc['label_count'] = df_cooc['labels'].apply(len)

# Nur Multi-Label Tweets analysieren (2+ Labels)
multi_label_tweets_cooc = df_cooc[df_cooc['label_count'] >= 2].copy()

print("=" * 70)
print("LABEL CO-OCCURRENCE ANALYSIS")
print("=" * 70)
print(f"\n✓ Gesamt Tweets: {len(df_cooc):,}")
print(f"✓ Multi-Label Tweets (2+ Labels): {len(multi_label_tweets_cooc):,}")
print(f"✓ Prozent Multi-Label: {100 * len(multi_label_tweets_cooc) / len(df_cooc):.1f}%")

# Zähle alle Label-Paare
label_pair_counts = Counter()

for labels in multi_label_tweets_cooc['labels']:
    # Für jeden Tweet: alle möglichen 2er-Kombinationen erstellen
    # sorted() stellt sicher, dass (A,B) und (B,A) als gleich behandelt werden
    pairs = combinations(sorted(labels), 2)
    for pair in pairs:
        label_pair_counts[pair] += 1

# Top 20 häufigste Label-Paare
top_pairs = label_pair_counts.most_common(20)

print(f"\n✓ Gefundene einzigartige Label-Paare: {len(label_pair_counts)}")
print(f"\n" + "-" * 70)
print("TOP 20 HÄUFIGSTE LABEL-KOMBINATIONEN")
print("-" * 70)
print(f"{'Rang':<6} {'Label-Paar':<55} {'Anzahl':<10}")
print("-" * 70)

for rank, (pair, count) in enumerate(top_pairs, 1):
    pair_str = f"{pair[0]} + {pair[1]}"
    print(f"{rank:<6} {pair_str:<55} {count:,}")

# Visualisierung als Heatmap
print("\n" + "=" * 70)
print("LABEL CO-OCCURRENCE HEATMAP")
print("=" * 70)

# Alle einzigartigen Labels aus den Multi-Label Tweets
all_labels = sorted(set(label for labels in multi_label_tweets_cooc['labels'] for label in labels))
n_labels = len(all_labels)
label_to_idx = {label: idx for idx, label in enumerate(all_labels)}

# Co-occurrence Matrix erstellen
cooccurrence_matrix = np.zeros((n_labels, n_labels), dtype=int)

for labels in multi_label_tweets_cooc['labels']:
    pairs = combinations(sorted(labels), 2)
    for label1, label2 in pairs:
        idx1, idx2 = label_to_idx[label1], label_to_idx[label2]
        cooccurrence_matrix[idx1, idx2] += 1
        cooccurrence_matrix[idx2, idx1] += 1  # Symmetrisch

# Diagonale = Anzahl Tweets mit diesem Label (für Kontext)
for labels in multi_label_tweets_cooc['labels']:
    for label in labels:
        cooccurrence_matrix[label_to_idx[label], label_to_idx[label]] += 1

# Heatmap erstellen
fig, ax = plt.subplots(figsize=(14, 12))

im = ax.imshow(cooccurrence_matrix, cmap='YlOrRd', aspect='auto')

# Achsenbeschriftung
ax.set_xticks(range(n_labels))
ax.set_yticks(range(n_labels))
ax.set_xticklabels(all_labels, rotation=45, ha='right', fontsize=9)
ax.set_yticklabels(all_labels, fontsize=9)

# Colorbar
cbar = plt.colorbar(im, ax=ax, shrink=0.8)
cbar.set_label('Co-occurrence Count', fontsize=11)

# Werte in Zellen anzeigen (nur für signifikante Werte)
for i in range(n_labels):
    for j in range(n_labels):
        value = cooccurrence_matrix[i, j]
        if value > 0 and i != j:  # Nur Off-Diagonal-Werte
            text_color = 'white' if value > cooccurrence_matrix.max() * 0.5 else 'black'
            ax.text(j, i, f'{value}', ha='center', va='center', 
                   fontsize=7, color=text_color)

ax.set_title('Label Co-occurrence Matrix\n(Multi-Label Tweets)', fontsize=14, fontweight='bold')
ax.set_xlabel('Label', fontsize=11)
ax.set_ylabel('Label', fontsize=11)

plt.tight_layout()
plt.show()

# Zusätzliche Statistiken
print("\n" + "=" * 70)
print("ZUSÄTZLICHE STATISTIKEN")
print("=" * 70)

# Labels die am häufigsten mit anderen kombiniert werden
label_combination_freq = Counter()
for labels in multi_label_tweets_cooc['labels']:
    for label in labels:
        label_combination_freq[label] += 1

print("\n✓ Labels die am häufigsten in Multi-Label Tweets vorkommen:")
print("-" * 50)
for label, count in label_combination_freq.most_common():
    pct = 100 * count / len(multi_label_tweets_cooc)
    print(f"  {label}: {count:,} ({pct:.1f}% der Multi-Label Tweets)")

# Durchschnittliche Anzahl verschiedener Partner-Labels pro Label
print("\n✓ Partner-Diversität pro Label:")
print("-" * 50)
for label in sorted(all_labels):
    partners = sum(1 for pair, count in label_pair_counts.items() 
                   if label in pair and count > 0)
    print(f"  {label}: kombiniert mit {partners} verschiedenen anderen Labels")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import re
from collections import Counter

def parse_label_names(label_str):
    """
    Parse the label_name string into a list of labels.
    The format is numpy-style: ['label1' 'label2'] instead of ['label1', 'label2']
    
    Parameters:
    -----------
    label_str : str
        String representation of labels in numpy array format
    
    Returns:
    --------
    list
        List of label strings
    """
    if isinstance(label_str, str):
        label_str = label_str.strip()
        if label_str.startswith('[') and label_str.endswith(']'):
            content = label_str[1:-1]
            items = re.findall(r"'([^']*)'" , content)
            return items
    return []


def prepare_data_for_training(df_train, df_validation, df_test, min_samples=180, show_plot=True):
    """
    Prepare datasets for training by removing labels with insufficient samples.
    
    This function performs the following steps:
    1. Counts the number of tweets per label in the training set
    2. Visualizes the label distribution using matplotlib
    3. Identifies labels with fewer than min_samples tweets
    4. Removes tweets containing only the underrepresented labels from all splits
    5. Updates the label vectors and label_name columns
    
    Parameters:
    -----------
    df_train : pd.DataFrame
        Training dataframe with 'text', 'label_name', and 'label' columns
    df_validation : pd.DataFrame
        Validation dataframe with same columns
    df_test : pd.DataFrame
        Test dataframe with same columns
    min_samples : int, default=180
        Minimum number of tweets required for a label to be kept
    show_plot : bool, default=True
        Whether to display the matplotlib visualization
    
    Returns:
    --------
    tuple of (pd.DataFrame, pd.DataFrame, pd.DataFrame, list)
        Filtered train, validation, test dataframes, and list of removed labels
    """
    
    # Make copies to avoid modifying original dataframes
    train = df_train.copy()
    val = df_validation.copy()
    test = df_test.copy()
    
    # Parse labels for all datasets
    train['parsed_labels'] = train['label_name'].apply(parse_label_names)
    val['parsed_labels'] = val['label_name'].apply(parse_label_names)
    test['parsed_labels'] = test['label_name'].apply(parse_label_names)
    
    # Count tweets per label in training set
    label_counts = Counter()
    for labels in train['parsed_labels']:
        for label in labels:
            label_counts[label] += 1
    
    # Sort labels by count for visualization
    sorted_labels = sorted(label_counts.items(), key=lambda x: x[1], reverse=True)
    labels_list = [item[0] for item in sorted_labels]
    counts_list = [item[1] for item in sorted_labels]
    
    # Identify labels to remove
    labels_to_remove = [label for label, count in label_counts.items() if count < min_samples]
    labels_to_keep = [label for label, count in label_counts.items() if count >= min_samples]
    
    print("=" * 60)
    print("LABEL DISTRIBUTION ANALYSIS")
    print("=" * 60)
    print(f"\nTotal unique labels: {len(label_counts)}")
    print(f"Minimum samples threshold: {min_samples}")
    print(f"Labels to keep: {len(labels_to_keep)}")
    print(f"Labels to remove: {len(labels_to_remove)}")
    
    if labels_to_remove:
        print(f"\nLabels being removed (< {min_samples} tweets):")
        for label in sorted(labels_to_remove, key=lambda x: label_counts[x], reverse=True):
            print(f"  - {label}: {label_counts[label]} tweets")
    
    # Visualization with matplotlib
    if show_plot:
        fig, ax = plt.subplots(figsize=(12, 8))
        
        # Create colors based on whether label will be kept or removed
        colors = ['#2ecc71' if count >= min_samples else '#e74c3c' for count in counts_list]
        
        # Create horizontal bar chart
        y_pos = np.arange(len(labels_list))
        bars = ax.barh(y_pos, counts_list, color=colors, edgecolor='black', linewidth=0.5)
        
        # Add threshold line
        ax.axvline(x=min_samples, color='#3498db', linestyle='--', linewidth=2, 
                   label=f'Threshold ({min_samples} tweets)')
        
        # Customize chart
        ax.set_yticks(y_pos)
        ax.set_yticklabels(labels_list)
        ax.invert_yaxis()  # Labels read top-to-bottom
        ax.set_xlabel('Number of Tweets', fontsize=12)
        ax.set_ylabel('Label', fontsize=12)
        ax.set_title('Label Distribution in Training Data\n(Green: Keep, Red: Remove)', fontsize=14)
        ax.legend(loc='lower right')
        
        # Add count annotations
        for i, (count, bar) in enumerate(zip(counts_list, bars)):
            ax.annotate(f'{count}', xy=(count + 20, bar.get_y() + bar.get_height()/2),
                       va='center', ha='left', fontsize=9)
        
        plt.tight_layout()
        plt.show()
        print(f"\n✓ Plot displayed")
    
    # Define the original label order (from the dataset)
    all_labels_ordered = [
        'arts_&_culture', 'business_&_entrepreneurs', 'celebrity_&_pop_culture',
        'diaries_&_daily_life', 'family', 'fashion_&_style', 'film_tv_&_video',
        'fitness_&_health', 'food_&_dining', 'gaming', 'learning_&_educational',
        'music', 'news_&_social_concern', 'other_hobbies', 'relationships',
        'science_&_technology', 'sports', 'travel_&_adventure', 'youth_&_student_life'
    ]
    
    # Get indices of labels to keep
    keep_indices = [i for i, label in enumerate(all_labels_ordered) if label in labels_to_keep]
    new_labels_ordered = [all_labels_ordered[i] for i in keep_indices]
    
    def filter_and_update_labels(df, labels_to_remove_set, keep_indices, new_labels_ordered):
        """
        Filter out rows where all labels are in labels_to_remove and update label vectors.
        """
        # Check if tweet has at least one label that will be kept
        def has_valid_label(parsed_labels):
            return any(label not in labels_to_remove_set for label in parsed_labels)
        
        # Filter rows
        mask = df['parsed_labels'].apply(has_valid_label)
        df_filtered = df[mask].copy()
        
        # Update label_name to only include kept labels
        def update_label_names(parsed_labels):
            kept = [label for label in parsed_labels if label not in labels_to_remove_set]
            return str(kept).replace(', ', ' ').replace(',', '')
        
        df_filtered['label_name'] = df_filtered['parsed_labels'].apply(update_label_names)
        
        # Update label vectors - parse old vector and create new one with only kept indices
        def update_label_vector(label_str):
            # Parse the vector string like '[0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0]'
            label_str = str(label_str).strip()
            if label_str.startswith('[') and label_str.endswith(']'):
                values = label_str[1:-1].split()
                values = [int(v) for v in values]
                # Keep only the values at keep_indices
                new_values = [values[i] for i in keep_indices]
                return '[' + ' '.join(map(str, new_values)) + ']'
            return label_str
        
        df_filtered['label'] = df_filtered['label'].apply(update_label_vector)
        
        # Drop the temporary parsed_labels column
        df_filtered = df_filtered.drop(columns=['parsed_labels'])
        
        return df_filtered
    
    labels_to_remove_set = set(labels_to_remove)
    
    # Apply filtering to all datasets
    train_filtered = filter_and_update_labels(train, labels_to_remove_set, keep_indices, new_labels_ordered)
    val_filtered = filter_and_update_labels(val, labels_to_remove_set, keep_indices, new_labels_ordered)
    test_filtered = filter_and_update_labels(test, labels_to_remove_set, keep_indices, new_labels_ordered)
    
    # Print summary
    print("\n" + "=" * 60)
    print("FILTERING RESULTS")
    print("=" * 60)
    print(f"\nTraining set:")
    print(f"  Before: {len(train)} tweets")
    print(f"  After:  {len(train_filtered)} tweets ({len(train) - len(train_filtered)} removed)")
    
    print(f"\nValidation set:")
    print(f"  Before: {len(val)} tweets")
    print(f"  After:  {len(val_filtered)} tweets ({len(val) - len(val_filtered)} removed)")
    
    print(f"\nTest set:")
    print(f"  Before: {len(test)} tweets")
    print(f"  After:  {len(test_filtered)} tweets ({len(test) - len(test_filtered)} removed)")
    
    print(f"\n✓ Label vector size reduced from {len(all_labels_ordered)} to {len(new_labels_ordered)}")
    print(f"\nRemaining labels ({len(new_labels_ordered)}):")
    for label in new_labels_ordered:
        print(f"  - {label}")
    
    return train_filtered, val_filtered, test_filtered, labels_to_remove


# Apply the prepare_data_for_training function
print("Loading preprocessed datasets...\n")

# Load the preprocessed datasets (from multi_label folder)
train_df = pd.read_parquet('../Data/multi_label/tweets_preprocessed_train.parquet')
val_df = pd.read_parquet('../Data/multi_label/tweets_preprocessed_validation.parquet')
test_df = pd.read_parquet('../Data/multi_label/tweets_preprocessed_test.parquet')

print(f"Loaded train: {len(train_df)} tweets")
print(f"Loaded validation: {len(val_df)} tweets")
print(f"Loaded test: {len(test_df)} tweets\n")

# Prepare data for training (remove labels with < 500 tweets)
train_filtered, val_filtered, test_filtered, removed_labels = prepare_data_for_training(
    train_df, val_df, test_df, 
    min_samples=500, 
    show_plot=True
)

# Save the filtered datasets to multi_label folder
train_filtered.to_parquet('../Data/multi_label/tweets_preprocessed_train.parquet', index=False)
val_filtered.to_parquet('../Data/multi_label/tweets_preprocessed_validation.parquet', index=False)
test_filtered.to_parquet('../Data/multi_label/tweets_preprocessed_test.parquet', index=False)

print("\n" + "=" * 60)
print("SAVED FILES")
print("=" * 60)
print("\n✓ Filtered datasets saved to:")
print("  - ../Data/multi_label/tweets_preprocessed_train.parquet")
print("  - ../Data/multi_label/tweets_preprocessed_validation.parquet")
print("  - ../Data/multi_label/tweets_preprocessed_test.parquet")
print("\n✓ Data preparation for training complete!")

---
## 3. Single-Label Dataset Creation

### 3.1 Intelligent Single-Label Assignment with Claude Haiku

For training a Single-Label classifier (needed in Lab 5), we need to convert multi-label samples to single-label. Instead of simply taking the first label, we use **Claude Haiku** to intelligently decide which label best represents the tweet's main topic.

**Strategy:**
1. **Single-label tweets**: Keep the label as-is
2. **Multi-label tweets**: Use Claude Haiku to analyze the tweet and select the most appropriate single label
3. **Caching**: Results are cached to avoid redundant API calls and reduce costs

**Output Files:**
- `../Data/single_label/tweets_single_label_train.parquet`
- `../Data/single_label/tweets_single_label_test.parquet`
- `../Data/single_label/tweets_single_label_validation.parquet`

In [ ]:
# ============================================================
# INTELLIGENTE SINGLE-LABEL ZUWEISUNG MIT CLAUDE HAIKU
# ============================================================
import anthropic
import json
import os
import hashlib
from pathlib import Path
from typing import List
from tqdm import tqdm
import time
import ast
import re

# Cache-Pfad für API-Ergebnisse (spart Kosten bei erneutem Ausführen)
SINGLE_LABEL_CACHE_PATH = Path("../Data/single_label/single_label_cache.json")

# Anthropic Client initialisieren
ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY", None)

if ANTHROPIC_API_KEY is None:
    print("⚠️ ANTHROPIC_API_KEY nicht gefunden!")
    print("   Bitte setze die Umgebungsvariable oder gib den Key hier ein:")
    print("   export ANTHROPIC_API_KEY='dein-api-key'")
    USE_LLM = False
else:
    client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
    USE_LLM = True
    print("✓ Anthropic Client initialisiert")

def parse_labels_for_single(value) -> List[str]:
    """Parse label_name column into consistent Python lists."""
    if isinstance(value, (list, np.ndarray)):
        return [str(v) for v in value]
    if isinstance(value, tuple):
        return [str(v) for v in value]
    if isinstance(value, str):
        value = value.strip()
        if value.startswith('[') and value.endswith(']'):
            inner = value[1:-1].strip()
            if not inner:
                return []
            inner = inner.replace("'", "").replace('"', '')
            labels = [l.strip() for l in inner.split() if l.strip()]
            return labels
        try:
            parsed = ast.literal_eval(value)
            if isinstance(parsed, (list, tuple)):
                return [str(v) for v in parsed]
        except (ValueError, SyntaxError):
            pass
        return [value] if value else []
    return [str(value)] if value else []

def load_single_label_cache() -> dict:
    """Lade gecachte Single-Label Entscheidungen."""
    if SINGLE_LABEL_CACHE_PATH.exists():
        with open(SINGLE_LABEL_CACHE_PATH, 'r', encoding='utf-8') as f:
            return json.load(f)
    return {}

def save_single_label_cache(cache: dict):
    """Speichere Cache auf Disk."""
    os.makedirs(SINGLE_LABEL_CACHE_PATH.parent, exist_ok=True)
    with open(SINGLE_LABEL_CACHE_PATH, 'w', encoding='utf-8') as f:
        json.dump(cache, f, ensure_ascii=False, indent=2)

def get_single_label_cache_key(tweet_id: str) -> str:
    """Erstelle einen eindeutigen Cache-Key nur aus der Tweet-ID."""
    return hashlib.md5(str(tweet_id).encode()).hexdigest()

def classify_with_haiku(tweet_id: str, text: str, labels: list, cache: dict) -> str:
    """
    Verwende Claude Haiku um das passendste Label auszuwählen.
    Mit Caching um API-Kosten zu minimieren.
    """
    cache_key = get_single_label_cache_key(tweet_id)
    
    # Prüfe Cache
    if cache_key in cache:
        return cache[cache_key]
    
    # Falls LLM nicht verfügbar, nimm erstes Label
    if not USE_LLM:
        return labels[0]
    
    # Erstelle Prompt
    system_prompt = """Du bist ein Experte für Tweet-Klassifizierung. 
Deine Aufgabe ist es, das EINE passendste Label für einen Tweet auszuwählen.
Antworte NUR mit dem gewählten Label, nichts anderes."""

    user_prompt = f"""Tweet: "{text}"

Mögliche Labels: {', '.join(labels)}

Welches Label passt am besten zu diesem Tweet? Antworte nur mit dem Label."""

    try:
        response = client.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=50,
            messages=[
                {"role": "user", "content": user_prompt}
            ],
            system=system_prompt
        )
        
        result = response.content[0].text.strip()
        
        # Validiere dass das Ergebnis ein gültiges Label ist
        if result in labels:
            cache[cache_key] = result
            return result
        else:
            # Falls LLM ungültiges Label zurückgibt, finde beste Übereinstimmung
            for label in labels:
                if label.lower() in result.lower():
                    cache[cache_key] = label
                    return label
            # Fallback: erstes Label
            cache[cache_key] = labels[0]
            return labels[0]
            
    except Exception as e:
        print(f"API Error: {e}")
        return labels[0]  # Fallback

def assign_single_labels_smart(df, cache: dict, desc: str = "Processing") -> list:
    """
    Weise jedem Sample das beste Single-Label zu.
    Verwendet LLM nur für Samples mit mehreren Labels.
    """
    single_labels = []
    api_calls = 0
    cache_hits = 0
    
    # Prüfe ob 'id' Spalte existiert
    has_id_column = 'id' in df.columns
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc=desc):
        labels = row['labels']
        # Verwende 'id' Spalte falls vorhanden, sonst den Index
        tweet_id = row['id'] if has_id_column else str(idx)
        
        if len(labels) == 1:
            # Nur ein Label - keine Entscheidung nötig
            single_labels.append(labels[0])
        else:
            # Mehrere Labels - LLM entscheidet (oder Cache)
            cache_key = get_single_label_cache_key(tweet_id)
            if cache_key in cache:
                cache_hits += 1
            else:
                api_calls += 1
            
            best_label = classify_with_haiku(tweet_id, row['text'], labels, cache)
            single_labels.append(best_label)
            
            # Speichere Cache regelmäßig
            if api_calls > 0 and api_calls % 100 == 0:
                save_single_label_cache(cache)
                print(f"\n  💾 Cache gespeichert ({api_calls} API calls, {cache_hits} cache hits)")
    
    print(f"\n  📊 Statistik: {api_calls} API calls, {cache_hits} cache hits, {len(df) - api_calls - cache_hits} single-label samples")
    return single_labels

print("\n✓ Single-Label Funktionen definiert")

### 3.2 Generate or Load Single-Label Datasets

Set `REGENERATE_SINGLE_LABELS = True` to regenerate the single-label datasets using Claude Haiku.
If the files already exist and regeneration is disabled, they will be loaded from disk.

In [ ]:
# ============================================================
# SINGLE-LABEL DATEN GENERIEREN ODER LADEN
# ============================================================
# Setze REGENERATE_SINGLE_LABELS = True um die LLM-Klassifizierung 
# erneut durchzuführen und die Dateien zu aktualisieren.

REGENERATE_SINGLE_LABELS = False  # ← Auf True setzen um LLM-Klassifizierung zu starten

# Definiere Pfade
SINGLE_LABEL_TRAIN_PATH = "../Data/single_label/tweets_single_label_train.parquet"
SINGLE_LABEL_TEST_PATH = "../Data/single_label/tweets_single_label_test.parquet"
SINGLE_LABEL_VALIDATION_PATH = "../Data/single_label/tweets_single_label_validation.parquet"

print("=" * 70)
print("SINGLE-LABEL DATASET ERSTELLUNG")
print("=" * 70)

# Lade die gefilterten Multi-Label Daten
print("\n📂 Lade gefilterte Multi-Label Daten...")
sl_train_df = pd.read_parquet('../Data/multi_label/tweets_preprocessed_train.parquet')
sl_test_df = pd.read_parquet('../Data/multi_label/tweets_preprocessed_test.parquet')
sl_val_df = pd.read_parquet('../Data/multi_label/tweets_preprocessed_validation.parquet')

# Parse Labels
sl_train_df['labels'] = sl_train_df['label_name'].apply(parse_labels_for_single)
sl_test_df['labels'] = sl_test_df['label_name'].apply(parse_labels_for_single)
sl_val_df['labels'] = sl_val_df['label_name'].apply(parse_labels_for_single)

print(f"✓ Training: {len(sl_train_df):,} Samples")
print(f"✓ Test: {len(sl_test_df):,} Samples")
print(f"✓ Validation: {len(sl_val_df):,} Samples")

# Prüfe ob gespeicherte Dateien existieren
train_exists = Path(SINGLE_LABEL_TRAIN_PATH).exists()
test_exists = Path(SINGLE_LABEL_TEST_PATH).exists()
val_exists = Path(SINGLE_LABEL_VALIDATION_PATH).exists()
all_files_exist = train_exists and test_exists and val_exists

if not REGENERATE_SINGLE_LABELS and all_files_exist:
    # ============================================================
    # OPTION 1: Lade gespeicherte Single-Label Dateien
    # ============================================================
    print("\n📂 Lade gespeicherte Single-Label Dateien...")
    
    sl_train_result = pd.read_parquet(SINGLE_LABEL_TRAIN_PATH)
    sl_test_result = pd.read_parquet(SINGLE_LABEL_TEST_PATH)
    sl_val_result = pd.read_parquet(SINGLE_LABEL_VALIDATION_PATH)
    
    print(f"✓ Training Set geladen: {len(sl_train_result):,} Samples")
    print(f"✓ Test Set geladen: {len(sl_test_result):,} Samples")
    print(f"✓ Validation Set geladen: {len(sl_val_result):,} Samples")
    
    print(f"\n✓ Single-Label Verteilung (Training):")
    print(sl_train_result['single_label'].value_counts())
    
    print("\n💡 Tipp: Setze REGENERATE_SINGLE_LABELS = True um die Dateien zu aktualisieren")

else:
    # ============================================================
    # OPTION 2: Generiere Single-Labels mit LLM
    # ============================================================
    if not all_files_exist:
        print("\n⚠️ Single-Label Dateien nicht gefunden - generiere mit LLM...")
    else:
        print("\n🔄 REGENERATE_SINGLE_LABELS = True - generiere Single-Labels mit LLM...")
    
    print("\n" + "=" * 70)
    print("INTELLIGENTE SINGLE-LABEL ZUWEISUNG MIT CLAUDE HAIKU")
    print("=" * 70)
    
    # Lade Cache
    cache = load_single_label_cache()
    print(f"✓ Cache geladen: {len(cache)} Einträge")
    
    # Konvertiere alle Datasets
    print("\n📌 Training Set:")
    sl_train_df['single_label'] = assign_single_labels_smart(sl_train_df, cache, desc="Training Set")
    
    print("\n📌 Test Set:")
    sl_test_df['single_label'] = assign_single_labels_smart(sl_test_df, cache, desc="Test Set")
    
    print("\n📌 Validation Set:")
    sl_val_df['single_label'] = assign_single_labels_smart(sl_val_df, cache, desc="Validation Set")
    
    # Speichere finalen Cache
    save_single_label_cache(cache)
    print(f"\n✓ Cache gespeichert: {len(cache)} Einträge")
    
    # Statistiken anzeigen
    print("\n" + "=" * 70)
    print("SINGLE-LABEL VERTEILUNG (Training Set)")
    print("=" * 70)
    print(sl_train_df['single_label'].value_counts())
    
    # Vergleich: Original erstes Label vs. LLM-Auswahl
    if USE_LLM:
        original_first = sl_train_df['labels'].apply(lambda x: x[0] if x else None)
        llm_selected = sl_train_df['single_label']
        changed = (original_first != llm_selected).sum()
        print(f"\n✓ LLM hat {changed:,} Labels anders gewählt als 'erstes Label' ({100*changed/len(sl_train_df):.1f}%)")
    
    # ============================================================
    # SPEICHERE SINGLE-LABEL DATEN
    # ============================================================
    print("\n" + "=" * 70)
    print("SPEICHERE SINGLE-LABEL DATASETS")
    print("=" * 70)
    
    # Erstelle Ordner falls nicht vorhanden
    os.makedirs('../Data/single_label', exist_ok=True)
    
    # Speichere Training Set
    sl_train_df.to_parquet(SINGLE_LABEL_TRAIN_PATH, index=False)
    print(f"✓ Training Set gespeichert: {SINGLE_LABEL_TRAIN_PATH}")
    
    # Speichere Test Set
    sl_test_df.to_parquet(SINGLE_LABEL_TEST_PATH, index=False)
    print(f"✓ Test Set gespeichert: {SINGLE_LABEL_TEST_PATH}")
    
    # Speichere Validation Set
    sl_val_df.to_parquet(SINGLE_LABEL_VALIDATION_PATH, index=False)
    print(f"✓ Validation Set gespeichert: {SINGLE_LABEL_VALIDATION_PATH}")
    
    sl_train_result = sl_train_df
    sl_test_result = sl_test_df
    sl_val_result = sl_val_df

print("\n" + "=" * 70)
print("✓ Single-Label Datasets erstellt/geladen!")
print("=" * 70)
print(f"\n📊 Output Dateien:")
print(f"  - {SINGLE_LABEL_TRAIN_PATH}")
print(f"  - {SINGLE_LABEL_TEST_PATH}")
print(f"  - {SINGLE_LABEL_VALIDATION_PATH}")